In [1]:
# Imports
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.merge import concatenate
import numpy as np
from google.colab import files


In [6]:
uploaded = files.upload()

Saving fens_01_shuf to fens_01_shuf


In [2]:
# list of input and output data
input_data = []
expected_output = []

# loading data
with open('fens_01_shuf', 'r') as file:
    for line in file:
        line = line[:-1].strip().split(' : ')
        fen = line[0]
        if fen not in input_data:
            input_data.append(fen)
            scores = line[1].split(' ')
            expected_output.append([float(i) for i in scores])

file.close              
print(len(input_data))                                

368428


In [3]:
# parsing fen to array of floats
def parse_fen(fen):
    
    input_data = []
    index = 0
    split = fen.split(' ')
    
    # peices on the board
    for i in range(len(fen)):
        if fen[i] == ' ':
            index += 1
            break
            
        if fen[i] == '/':
            index += 1
            continue
            
        if fen[i] in chess_pieces:
            input_data.append(chess_pieces[fen[i]])
            index += 1
            
        else:
            for i in range(int(fen[i])):
                input_data.append(chess_pieces['.'])
            index += 1
    
    # white's/black's move
    if fen[index] == 'w':
        input_data.append(1)
    else:
        input_data.append(-1)
        
    # castling
    # skip space
    index += 1
    if fen[index] == '-':
        input_data.append(0)
        input_data.append(0)
        input_data.append(0)
        input_data.append(0)
        index += 4
    else:
        castle = split[2]
        if 'K' in castle:
            input_data.append(1)
        else:
            input_data.append(0)
        if 'Q' in castle:
            input_data.append(1)
        else:
            input_data.append(0)
        if 'k' in castle:
            input_data.append(-1)
        else:
            input_data.append(0)
        if 'q' in castle:
            input_data.append(-1)
        else:
            input_data.append(0)
     
    # en passant
    if split[3] != '-':
        if split[1] == 'w':
            input_data.append(1)
        else:
            input_data.append(-1)
    else:
        input_data.append(0)
        
    
        
    return np.asarray(input_data[:64]).reshape((8,8, 12)), np.asarray(input_data[64:]) 

chess_pieces = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
}

In [4]:
# parse fens for every position
inputs = []
input_boards = []
input_states = []
for i in input_data:
    input_board, input_state = parse_fen(i)
    input_boards.append(input_board)
    input_states.append(input_state)

input_boards = np.asarray(input_boards)
input_states = np.asarray(input_states)

In [8]:
# Build model
# Model input is 1D array of length 70.
# First part of the model is CNN, where first 64 input values are reshaped to 8x8 2D array.
# Second part is normal ANN, where last six input values are appended to output of CNN.
board_input = keras.Input(shape=(8, 8, 12), name="board")
state_input = keras.Input(shape=(6,), name='state')
x = layers.Conv2D(filters=512, kernel_size=4, padding="same", activation="relu", 
                  input_shape=(8,8,12))(board_input)
x = layers.Conv2D(filters=512, kernel_size=3, padding="same", activation="relu")(x)                  
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=512, kernel_size=2, padding="same", activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=512, kernel_size=1, padding="same", activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
merge = concatenate([x, state_input])
x = layers.Dense(512, activation="relu")(merge)
#x = layers.Dense(512, activation="relu")(x)
board_output = layers.Dense(3, activation='softmax')(x)


model = keras.Model(inputs=[board_input, state_input], outputs=board_output, name="prediction_conv")


In [9]:
# Compile  model.
model.compile(
  optimizer=optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [10]:
# train model

model.fit(
  [input_boards, input_states],
  np.asarray(expected_output),
  epochs=12,
  batch_size=256,
  validation_split=0.2,
  #callbacks=[es],  
)

Epoch 1/12
1152/1152 [==============================] - 63s 53ms/step - loss: 0.6780 - accuracy: 0.7044 - val_loss: 0.5964 - val_accuracy: 0.7492
Epoch 2/12
1152/1152 [==============================] - 59s 52ms/step - loss: 0.4975 - accuracy: 0.7936 - val_loss: 0.6557 - val_accuracy: 0.7329
Epoch 3/12
1152/1152 [==============================] - 59s 52ms/step - loss: 0.4042 - accuracy: 0.8364 - val_loss: 0.4487 - val_accuracy: 0.8223
Epoch 4/12
1152/1152 [==============================] - 60s 52ms/step - loss: 0.3351 - accuracy: 0.8679 - val_loss: 0.4336 - val_accuracy: 0.8307
Epoch 5/12
1152/1152 [==============================] - 60s 52ms/step - loss: 0.2793 - accuracy: 0.8903 - val_loss: 0.4289 - val_accuracy: 0.8392
Epoch 6/12
1152/1152 [==============================] - 59s 52ms/step - loss: 0.2395 - accuracy: 0.9071 - val_loss: 0.4533 - val_accuracy: 0.8404
Epoch 7/12
1152/1152 [==============================] - 59s 52ms/step - loss: 0.2048 - accuracy: 0.9213 - val_loss: 0.4521 -

In [ ]:
# evaluate model

model.evaluate(
    X_test,
    y_test
)

285/285 [==============================] - 1s 4ms/step - loss: 0.5868 - accuracy: 0.8046


[0.5867630243301392, 0.8045717477798462]

In [ ]:
# predictions
positions = []
positions.append(parse_fen('r1b1r1k1/pp1n2p1/1qp1pp1p/3p4/1PPP4/P1QBP1B1/5PPP/R4R1K w - - 4 19', values))

predictions = model.predict(positions)
print(np.argmax(predictions, axis=1))

[1]


In [ ]:
# save model
model.save_weights('model.h6')